# OpenSNP

## Packages and functions

In [8]:
import os
import sys
import pickle
import json
import subprocess
import pandas as pd
import numpy as np

import requests
from lxml import html

## Uploads

In [ ]:
url = "https://opensnp.org/genotypes?page={}"

# Get the number of pages
page = requests.get(url.format(1))
tree = html.fromstring(page.content)
num_pages = int(tree.xpath('//div[@class="pagination"]/ul/li/a/text()')[-1])

# Get the number of genotypes per page
page = requests.get(url.format(1))
tree = html.fromstring(page.content)
num_genotypes = len(tree.xpath('//div[@class="genotype"]/div[@class="genotype-name"]/a/text()'))

# Get the number of genotypes
num_genotypes = num_pages * num_genotypes

# Get the number of users
page = requests.get(url.format(1))
tree = html.fromstring(page.content)
num_users = len(tree.xpath('//div[@class="genotype"]/div[@class="genotype-user"]/a/text()'))



In [5]:
url = "https://opensnp.org/genotypes?page={}"

all_files = []

# Get table of file information
req = requests.session()
for i in range(1, 400):
    page = req.get(url.format(i), verify=False)
    tree = html.fromstring(page.text)
    
    # Get file name
    file_names = [x.lstrip('../data/') for x in tree.xpath("//table[@class='table table-hover']//tr//td[5]/a/@href")]
    if len(file_names) == 0:
        break

    # Get file ID
    fileIDs = [x.lstrip() for x in tree.xpath("//table[@class='table table-hover']//td[2]//text()[1]")]
    
    # Get time uplodaded
    times = [x.lstrip() for x in tree.xpath("//table[@class='table table-hover']//td[3]//text()[1]")]

    # Get user ID
    nicknames = [x.lstrip() for x in tree.xpath("//table[@class='table table-hover']//td[1]//text()[1]") if not '\n' in x]

    assert (len(file_names) == len(fileIDs) == len(times) == len(nicknames))

    for j in range(0, len(file_names)):
        assert(fileIDs[j] == file_names[j].split('.')[2])
        all_files.append({
            'nickname': nicknames[j], \
            'userID': file_names[j].split('.')[0], \
            'fileID': fileIDs[j], \
            'filename': file_names[j], \
            'time': times[j], \
            'download': 'https://opensnp.org/data/' + file_names[j]
        })

/Users/clu/softwoare/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opensnp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/clu/softwoare/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opensnp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/clu/softwoare/miniconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opensnp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/clu/softwoar

In [11]:
uploads = pd.DataFrame({
    'nickname': [x['nickname'] for x in all_files], \
    'userID': [x['userID'] for x in all_files], \
    'fileID': [x['fileID'] for x in all_files], \
    'filename': [x['filename'] for x in all_files], \
    'time': [x['time'] for x in all_files], \
    'download': [x['download'] for x in all_files]
})
uploads.to_csv('data/opensnp_uploads.csv', index=False)